In [ ]:
import os
import math
import string
from PIL import Image, ImageDraw, ImageFont
from google.colab import drive
from google.colab import files  # Biblioteca para download automático

# 1. Montar o Google Drive
# Garante acesso aos arquivos na nuvem
drive.mount('/content/drive', force_remount=True)

def criar_colagem_vertical_700dpi(diretorio_alvo, nome_saida='colagem_vertical_700dpi.jpg'):
    """
    Cria uma colagem científica empilhando imagens verticalmente (uma coluna).
    Mantém a resolução de 700 DPI e redimensiona todas para a maior dimensão encontrada.
    """

    # Limpeza e verificação do caminho
    diretorio_alvo = diretorio_alvo.strip()
    if not os.path.exists(diretorio_alvo):
        print(f"❌ ERRO: Diretório não encontrado: {diretorio_alvo}")
        return

    # Extensões de imagem permitidas
    extensoes = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif')
    arquivos = [f for f in os.listdir(diretorio_alvo) if f.lower().endswith(extensoes)]
    arquivos.sort(key=lambda x: x.lower()) # Ordenação alfabética para consistência (A, B, C...)

    if not arquivos:
        print("⚠️ Nenhuma imagem encontrada.")
        return

    # Selecionar até 24 imagens (limite de segurança, pode ser aumentado)
    lista_img = arquivos[:24]
    print(f"Processando {len(lista_img)} imagens em formato VERTICAL (700 DPI)...")

    # 2. Determinar a resolução máxima (Padronização)
    imagens_obj = []
    max_w = 0
    max_h = 0

    for nome in lista_img:
        caminho = os.path.join(diretorio_alvo, nome)
        try:
            # Abre e força RGB para evitar erros com imagens em tons de cinza ou CMYK
            img = Image.open(caminho).convert('RGB')
            imagens_obj.append(img)

            # Detecta as maiores dimensões para equalizar o tamanho
            if img.width > max_w: max_w = img.width
            if img.height > max_h: max_h = img.height
        except Exception as e:
            print(f"Erro ao ler {nome}: {e}")

    if not imagens_obj:
        return

    # Configurar tamanho da célula (baseado na maior imagem)
    largura_celula = max_w
    altura_celula = max_h
    num_imgs = len(imagens_obj)

    # --- ALTERAÇÃO PRINCIPAL: LÓGICA VERTICAL ---
    # A largura total é apenas 1 célula.
    # A altura total é a soma de todas as células verticalmente.
    largura_total = largura_celula
    altura_total = num_imgs * altura_celula

    print(f"Resolução do Canvas Final: {largura_total}x{altura_total} pixels")

    # Criar Canvas Branco
    # OBS: Se a altura for muito grande (> 65000 pixels), alguns visualizadores podem ter problemas,
    # mas o formato JPEG suporta até 65535px.
    canvas = Image.new('RGB', (largura_total, altura_total), (255, 255, 255))

    # Configurar Fonte (Proporcional à altura da imagem individual)
    tamanho_fonte = int(altura_celula * 0.08) # Reduzi levemente para 8% para ficar mais discreto
    try:
        # Tenta carregar uma fonte sem serifa padrão do sistema Linux/Colab
        fonte = ImageFont.truetype("LiberationSans-Regular.ttf", tamanho_fonte)
    except:
        fonte = ImageFont.load_default()

    alfabeto = string.ascii_uppercase

    # 3. Montagem Vertical
    for i, img in enumerate(imagens_obj):
        # Lógica vertical:
        # X é sempre 0 (alinhado à esquerda)
        # Y desce conforme o índice i aumenta
        x = 0
        y = i * altura_celula

        # Redimensionamento suave (LANCZOS) para padronizar o tamanho
        # Isso garante que a coluna fique perfeitamente alinhada
        img_final = img.resize((largura_celula, altura_celula), Image.LANCZOS)
        canvas.paste(img_final, (x, y))

        # Etiquetagem (A, B, C...)
        draw = ImageDraw.Draw(canvas)
        letra = alfabeto[i] if i < 26 else f"Z{i}"

        # Margem de 3% em relação ao topo/esquerda da célula atual
        margem = int(largura_celula * 0.03)
        pos_txt = (x + margem, y + margem)

        # Desenhar fundo preto atrás da letra para contraste
        bbox = draw.textbbox(pos_txt, letra, font=fonte)
        draw.rectangle((bbox[0]-10, bbox[1]-10, bbox[2]+10, bbox[3]+10), fill="black")
        draw.text(pos_txt, letra, fill="white", font=fonte)

    # 4. Salvamento e Exportação
    caminho_final = os.path.join(diretorio_alvo, nome_saida)

    # dpi=(700, 700): Define metadados de impressão
    # subsampling=0: Máxima qualidade de cor (4:4:4)
    canvas.save(caminho_final, format='JPEG', quality=100, subsampling=0, dpi=(700, 700))

    print(f"✅ Arquivo Vertical salvo: {caminho_final}")
    print("⏳ Iniciando download automático...")

    # 5. Download Automático para a máquina local
    files.download(caminho_final)

# --- EXECUÇÃO ---
# Substitua pelo caminho da sua pasta no Drive
pasta_alvo = '/content/drive/MyDrive/pesquisa_mestrado_2025/unir_fotos/IMAGE 2'
criar_colagem_vertical_700dpi(pasta_alvo)

Mounted at /content/drive
Processando 23 imagens em formato VERTICAL (700 DPI)...
Resolução do Canvas Final: 2726x31579 pixels
✅ Arquivo Vertical salvo: /content/drive/MyDrive/pesquisa_mestrado_2025/unir_fotos/IMAGE 2/colagem_vertical_700dpi.jpg
⏳ Iniciando download automático...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>